In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# Imports
# Natural Language Processing libraries, initiations and functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import re


# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-zA-Z]',' ', text.lower())
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 1 and not word in stop_words])



# Cvec, Standard
cvec = CountVectorizer(analyzer = "word",
                       min_df = 2,
                       preprocessor = preprocess,
                       stop_words = 'english') 
# Cvec DF
#df_words = pd.DataFrame(cvec.fit_transform(df['doc_column']).todense(), 
#                        columns=cvec.get_feature_names())

In [4]:
df_canna = pd.read_csv('./../../cannabis.csv')
df_canna.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [5]:
#df_canna['Starting_dex'] = df_canna.index
df_canna.dropna(inplace = True)

In [7]:
mask = df_canna['Type'] != 'hybrid'
df_canna = df_canna[mask]

In [8]:
df_canna.reset_index(inplace = True)

In [9]:
df_canna['Type'] = df_canna['Type'].map( lambda x: {'indica': 1,
 'sativa': 0}[x])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_effects = pd.DataFrame(cvec.fit_transform(df_canna['Effects']).todense(), 
                        columns=cvec.get_feature_names())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [11]:
df_flavors = pd.DataFrame(cvec.fit_transform(df_canna['Flavor']).todense(), 
                        columns=cvec.get_feature_names())

In [12]:
dict_E = {}
for col in df_effects.columns:
    dict_E[col] = ('E_' + col)

df_effects.rename(columns=dict_E, inplace = True)

In [13]:
dict_F = {}
for col in df_flavors.columns:
    dict_F[col] = ('F_' + col)

df_flavors.rename(columns=dict_F, inplace = True)

In [15]:
df = pd.concat([df_canna.loc[:,["Type", "Rating"]], df_effects
                #, df_flavors
               ], axis = 1)

In [20]:
y = df['Type']
X = df.drop(['Type'], axis = 1)

In [21]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

#import sklearn.metrics as metrics

#from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

print('Train:', model.score(X_train,y_train))
print('Test: ',  model.score(X_test,y_test))
np.average(cross_val_score(model, X, y, cv=3))

Train: 0.851985559566787
Test:  0.8700361010830325


0.847459654776728

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

model = RandomForestClassifier()
model.fit(X_train, y_train)

print('Train:', model.score(X_train,y_train))
print('Test: ',  model.score(X_test,y_test))
np.average(cross_val_score(model, X, y, cv=3))

Train: 0.9530685920577617
Test:  0.7978339350180506


0.8230913840669937

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

model = KNeighborsClassifier()
model.fit(X_train, y_train)

print('Train:', model.score(X_train,y_train))
print('Test: ',  model.score(X_test,y_test))
np.average(cross_val_score(model, X, y, cv=3))

Train: 0.8724428399518652
Test:  0.8267148014440433


0.8267047535340217

In [25]:
y_hat = model.predict(X_test)

In [26]:
pd.Series(y_hat).value_counts()

1    187
0     90
dtype: int64

In [27]:
df_canna['Type'].value_counts()

1    680
0    428
Name: Type, dtype: int64

Can we turn this into a %-age problem?
Can we cut down and isolate the values that're the most important to determining the best quality?
I'm in a weird argument with myself in which, TECHNICALLY I made a better model and thus satisfied today's requirements.  BUT I HAVE SUCH MANY OTHER BETTER IDEAS.  BUT ALSO HOLY BUTTS I NEED TO TAKE THE REST OF TODAY (AND MAYBE TOMORROW) EASY.